<a href="https://colab.research.google.com/github/pepedp/COVID-19/blob/master/ImagenesTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![texto alternativo](https://soyoungpark.github.io/assets/images/tensorflowtraffic/tf.png)

REFERENCIA - TENSORFLOW: 

 - https://www.tensorflow.org/tutorials/keras/classification 


REFERENCIA - KERAS: 

 - https://torres.ai/deep-learning-inteligencia-artificial-keras/

 - https://torres.ai/deep-learning-inteligencia-artificial-keras-2a-parte/





**Descripción del Ejemplo**

El laboratorio consiste en crear una Inteligencia Artificial capas de categorizar imagenes

In [ ]:
#@title Librerias y Funciones 
#Librerias  
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

import os
import zipfile
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator #Procesado de imagenes 
%matplotlib inline

from google.colab import auth
auth.authenticate_user()

#from google.colab import drive
#drive.mount('/content/drive')

#Habilitar TensroBoard 
import datetime
%load_ext tensorboard
!rm -rf ./logs/ 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


#FUNCIONES 
#Funcion para desplegar imagenes 
clasificacion=[]
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
 
  plt.imshow(img, cmap=plt.cm.binary)
 
  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
 
  plt.xlabel("{} {:2.0f}% ({})".format(clasificacion[predicted_label],
                                100*np.max(predictions_array),
                                clasificacion[true_label]),
                                color=color)
 
def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
 
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

#Funcion para ajustar registro grafico barras 
def autolabel(rects):
  for rect in rects:
      height = rect.get_height()
      plt.annotate('{}'.format(height),
                  xy=(rect.get_x() + rect.get_width() / 2, height),
                  xytext=(0, 3),  # 3 points vertical offset
                  textcoords="offset points",
                  ha="center", va='bottom')

# Procesar Imagenes

In [ ]:
#@title Cargar imagenes 
dataset_numeros = False #@param {type:"boolean"}
dataset_transporte = False #@param {type:"boolean"}
imag_mostrar = 20 #@param {type:"slider", min:5, max:100, step:5}
 
if (dataset_numeros):
  nombre_dataset="mnist"
  dataset_imagenes = tf.keras.datasets.mnist #Dataset que ya vienen en la libreria Tensoflow.keras
  clasificacion = ['Cero','Uno', 'Dos', 'tres', 'Cuatro', 
                   'Cinco','Seis', 'Siete', 'Ocho', 'Nueve']
elif(dataset_transporte):
  nombre_dataset="cifar10"
  dataset_imagenes = tf.keras.datasets.cifar10 
  clasificacion = ['avión', 'coche', 'pájaro', 'gato', 'ciervo', 
                   'perro', 'rana', 'caballo', 'barco', 'camión']
else:
  nombre_dataset="fashion_mnist"
  dataset_imagenes=tf.keras.datasets.fashion_mnist
  clasificacion = ['Camiseta/top', 'Pantalón', 'Suéter', 'Vestido', 'Saco',
                   'Sandalia', 'Camisa', 'Sneaker', 'Bolso', 'Botín']
 
  """clasificacion = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']"""
 
(x_train, y_train), (x_test, y_test) = dataset_imagenes.load_data()
tipo_imagen="a color (RBG)" if x_train.ndim>3 else "en escala de grises"
y_train,y_test=y_train.flatten(),y_test.flatten()

print("DATASET: {}".format(nombre_dataset) )
print("""ENTRENAMIENTO: \nx_train: Dimension del array[{}] | Shape:[{}] 
y_train: Dimension del array[{}] | Shape:[{}] 
\nTEST \nx_test: Dimension del array[{}] | Shape:[{}] 
y_test: Dimension del array[{}] | Shape:[{}] 
\nRESUMEN
 - Tamaño de cada imagen {}x{} píxeles {}
 - {:,.0f} imagenes de entrenamieno
 - {:,.0f} imagenes de test 
 - y_train & y_test contienen la categoria que pertenece la imagen (10 categorias)
""".format(x_train.ndim,x_train.shape,
           y_train.ndim,y_train.shape,
           x_test.ndim,x_test.shape,
           y_test.ndim,y_test.shape,
           x_train.shape[1],x_train.shape[2],tipo_imagen,
           x_train.shape[0],
           x_test.shape[0]           
           ))

#imag_mostrar=200
alto, ancho = 10,imag_mostrar/2
print("Primeras {} imagenes".format(imag_mostrar))
fig=plt.figure(figsize=(alto,ancho))
for i in range(imag_mostrar):
    ax=fig.add_subplot(imag_mostrar/5,5,i+1)
    ax.xaxis.label.set_color("white")
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(clasificacion[y_train[i]] )
plt.show()
print("Respuesta: {}".format(y_train[:imag_mostrar]))

In [ ]:
x_train.shape

In [ ]:
#@title Datos de la imagen 
Numero_imagen =  1#@param {type:"integer"}
if Numero_imagen!=0:
  np.set_printoptions(edgeitems=5, linewidth=1000, 
      formatter=dict(float=lambda x: "%.3g" % x))
  print("Respuesta: {}".format(y_train[Numero_imagen-1:Numero_imagen]) )
  print(x_train[Numero_imagen-1])
else:
  print("Imagen cero no existe, debe ser mayor a cero")

In [ ]:
#@title Redimensionar Dataset
x_train, x_test = x_train / 255.0, x_test / 255.0

**Arquitectura de la red Neuronal**

 - RNA: Arquitectura de Red Neuronal Artificial, 
 - RNC: Arquitectura de Red Neuronal Convolucional

In [ ]:
#@title Crear Red Neuronal
RNA = False #@param {type:"boolean"}
RNC = True #@param {type:"boolean"}


if RNC==True and RNA==False:
  modelo = tf.keras.models.Sequential()
  if x_train.ndim<4:
    modelo.add(tf.keras.layers.Conv1D(filters=28, kernel_size=3, padding="same", activation="relu", 
                                      input_shape=[x_train.shape[1], x_train.shape[2]]) )
    modelo.add(tf.keras.layers.Conv1D(filters=28, kernel_size=3, padding="same", activation="relu"))
    modelo.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='valid'))
    modelo.add(tf.keras.layers.Conv1D(filters=56, kernel_size=3, padding="same", activation="relu"))
    modelo.add(tf.keras.layers.Conv1D(filters=56, kernel_size=3, padding="same", activation="relu"))
    modelo.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='valid'))
    modelo.add(tf.keras.layers.Flatten())
    modelo.add(tf.keras.layers.Dense(units=128, activation='relu'))
    modelo.add(tf.keras.layers.Dropout(0.3))
    modelo.add(tf.keras.layers.Dense(units=10, activation='softmax'))
    modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    modelo.summary()
    
  else:
    modelo.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", 
                                      input_shape=[x_train.shape[1], x_train.shape[2],x_train.shape[3]]) )  
    #modelo.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[32, 32, 3]) )
    modelo.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
    modelo.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
    modelo.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
    modelo.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
    modelo.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
    modelo.add(tf.keras.layers.Flatten())
    modelo.add(tf.keras.layers.Dense(units=300, activation='relu'))
    modelo.add(tf.keras.layers.Dropout(0.2))
    modelo.add(tf.keras.layers.Dense(units=300, activation='relu'))
    modelo.add(tf.keras.layers.Dropout(0.2))
    modelo.add(tf.keras.layers.Dense(units=200, activation='relu'))
    #modelo.add(tf.keras.layers.Dropout(0.3))
    modelo.add(tf.keras.layers.Dense(units=10, activation='softmax'))
    modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    modelo.summary()
  
elif RNC==False and RNA==True:
  modelo = tf.keras.models.Sequential() #inicializamos el modelo (Creamos el objeto modelo) 
  if x_train.ndim<4:
    modelo.add(tf.keras.layers.Flatten(input_shape=(x_train.shape[1], x_train.shape[2]) )) #784 Variables de entrada espera la Red Neuronal (28*28=784)
  else:
    modelo.add(tf.keras.layers.Flatten(input_shape=(x_train.shape[1], x_train.shape[2],x_train.shape[3]) ))
  modelo.add(tf.keras.layers.Dense(128, activation='relu')) #128 Neuronas, función de activación Relu
  modelo.add(tf.keras.layers.Dropout(0.3))  #Neuronas que desactivamos para el entrenamiento (20%) evitar overfitting
  modelo.add(tf.keras.layers.Dense(10)) #La salida = 10; que es igual a la cantidad de clasificaciones del dataset
  modelo.add(tf.keras.layers.Dense(units=10, activation='softmax')) #Función de salida para la clasificación y probabilidad
  modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
  modelo.summary()

else:
  print ("Solo seleccionar solo una arquitectura")

In [ ]:
#@title Entrenar a la Red Neuronal
Entrenamiento =  5#@param {type:"number"}
Validacion = True #@param {type:"boolean"}

!rm -rf ./logs/ #Limpiar datos TensorBoard 

if (Validacion):
  modelo.fit(x_train, y_train, epochs=Entrenamiento,validation_data=(x_test, y_test),callbacks=[tensorboard_callback] )
else:
  modelo.fit(x_train, y_train, epochs=Entrenamiento,callbacks=[tensorboard_callback])

"""#GRAFICO
tipo_arq="RNA" if (RNA) else "RNC"
plt.figure(figsize=(10,6), facecolor="#E8E1E0")
plt.axes(facecolor='#E8E1E0')
plt.plot(modelo.history.history['sparse_categorical_accuracy'])  
plt.plot(modelo.history.history['val_sparse_categorical_accuracy'],'g')  
plt.xticks(np.arange(0, Entrenamiento+1, 1)) 
plt.xlabel("Numero de entrenamientos")  
plt.ylabel("Exactitud")  
plt.title("Resultados entrenamiento de la {}".format(tipo_arq) )  
plt.grid(color='white')
plt.legend(['Imagenes Entrenamiento','Imagenes Test'])"""


In [ ]:
#@title Resultados del Entrenamiento 
%tensorboard --logdir logs/fit

**Resultados de la Red Neuronal con los datos de Test**

In [ ]:
#@title Ejecutar Examen
predictions=modelo.predict(x_test) #prediccion para imagenes
y_pred_proba = modelo.predict_proba(x_test)  #prediccion probabilidad 
metricas=modelo.evaluate(x_test,  y_test, verbose=2)
print("""Exactitud: [{}]""".format(np.round(metricas[1], decimals=4)) )
print("\nMATRIZ DE CONFUSION: ")

#MATRIZ DE CONFUSION
y_pred = modelo.predict_classes(x_test)
mc=pd.crosstab(y_test, y_pred)
mc.columns=clasificacion

#Generar la exactitud de la Matriz de Confusion 
prob_mc,i=[],0
for i in range (len(mc.iloc[:].values)):
  prob_mc.append( (mc.iloc[i,i:i+1].values/sum(mc.iloc[i].values))*100 )

#Union de Dataframes
matriz_confusion=pd.concat([pd.DataFrame(clasificacion,columns=["categoría"]),
                mc,pd.DataFrame(prob_mc,columns=["Exactitud(%)"])],axis=1)

#GRAFICO
tipo_arq="RNA" if (RNA) else "RNC"
plt.figure(figsize=(16,3), facecolor="#E8E1E0")
#plt.xticks(np.arange(0, Entrenamiento+1, 1)) 
plt.title("Matriz de Confusión {}".format(tipo_arq) )  
plt.ylabel("Exactitud(%)")  
autolabel(plt.bar(clasificacion,np.round(np.array(prob_mc).flatten(),decimals=0),
                color="#5AADF2") )


matriz_confusion

In [ ]:
#@title Guardar resultados del examen 
path,nom_archivo1,nom_archivo2="/content/drive/My Drive/Dataset/","R_ImagenesTF.csv","MC_ImagenesTF.csv"
pd.DataFrame(y_pred_proba,y_test).to_csv(path+nom_archivo1)
matriz_confusion.to_csv(path+nom_archivo2)
print("""Archivo Detalle generado: {}{} 
\nArchivo Matriz confusión generado: {}{}""".format(
    path,nom_archivo1,
    path,nom_archivo2) )

In [ ]:
#@title Procesar una Imagen 
posicion_imagen =  192#@param {type:"integer"}
i = posicion_imagen
plt.figure(figsize=(6,3),facecolor="#E8E1E0")
plt.subplot(1,2,1)
plot_image(i, predictions[i], y_test, x_test)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i], y_test)
plt.show()

In [ ]:
#@title Procesar varias imagenes 
num_rows =  10#@param {type:"integer"}
ini_imagen =  0#@param {type:"integer"}
num_cols=5

num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows),facecolor="#E8E1E0")
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i+ini_imagen, predictions[i+ini_imagen], y_test, x_test)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i+ini_imagen, predictions[i+ini_imagen], y_test)
plt.tight_layout()
plt.show()

In [ ]:
#@title Guardar archivo 
"""import sys
if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()"""

#archivo=path+"ARQ_ImagenesTF.json"
archivo=path+"ARQ_ImagenesTF"
modelo_json = modelo.to_json()
with open("{}.json".format(archivo), "w") as json_file:
    json_file.write(modelo_json)
print("Archivo creado (Arquitectura): {}.json".format(archivo))

#Guardar los pesos de la Red Neuronal
modelo.save_weights("{}_pesos.h5".format(archivo))
print("Archivo creado (Weights): {}_pesos.h5".format(archivo))

# Aprendizaje por Transferencia 


In [ ]:
#@title Cargar Dataset
#CARGAR DATOS DESDE GCS
path_storage="001_storage/Datasets/cats_and_dogs/"
archivo = "cats_and_dogs.zip"
!gsutil cp gs://{path_storage}{archivo} /content/{archivo}

#DESCOMPRIMIR DATASET 
dataset_path = "./{}".format(archivo)
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()

In [ ]:
#@title Redimensionar los datos 
#https://keras.io/api/preprocessing/image/ 

#VARIABLES INICIALES
imagen_size = 224
batch_size = 10

#DIRECTORIO DE LOS DATOS 
dataset_path_new = "./cats_and_dogs_filtered/"
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")


#ESCALAR LAS IMAGENES 
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)


#ESTANDARIZAR TAMAÑO DE LAS IMAGENES 
print("Imagenes de entrenamiento detectadas:")
imagenes_train = data_gen_train.flow_from_directory(
    train_dir, 
    target_size=(imagen_size,imagen_size), 
    batch_size=batch_size, 
    class_mode="categorical")


print("\nImagenes de validación detectadas:")
imagenes_test = data_gen_valid.flow_from_directory(
    validation_dir,
    target_size=(imagen_size,imagen_size), 
    batch_size=batch_size, 
    class_mode="categorical")


In [ ]:
#@title Modificar Arquitectura Base Red Neuronal
#CARGAR MODELO BASE 
tamanio_imagen = (imagen_size, imagen_size, 3)
modelo_base = tf.keras.applications.DenseNet201(
    input_shape=tamanio_imagen,
    include_top=False, 
    weights="imagenet")

#DEFINIR CAPAS QUE SE VAN AGREGAR AL MODELO BASE
modelo_base.trainable = False  #Congelar modelo Base
modelo_head = tf.keras.layers.GlobalAveragePooling2D()(modelo_base.output) #Agregamos capa AveragePooling 
modelo_head = tf.keras.layers.Dense(units=2, activation='softmax')(modelo_head)

#AGREGAR CAPAS AL MODELO BASE
modelo = tf.keras.models.Model(inputs=modelo_base.input, outputs=modelo_head)
modelo.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss="categorical_crossentropy", metrics=["accuracy"])

modelo.summary() #Resumen de la nueva arquitectura Red Neuronal  

In [ ]:
#@title
tamanio_imagen = (224, 224, 3)
modelo_base = tf.keras.applications.DenseNet201(
    input_shape=tamanio_imagen,
    include_top=True, 
    weights="imagenet")

In [ ]:
#@title Entrenar a la Red Neuronal
Entrenamiento =  3#@param {type:"number"}

!rm -rf ./logs/ #Limpiar datos TensorBoard 
modelo.fit(imagenes_train,
           epochs=Entrenamiento, 
           validation_data=imagenes_test,
           callbacks=[tensorboard_callback])

In [ ]:
#@title Evaluar exactitud del modelo
%tensorboard --logdir logs/fit
set_datos=imagenes_test
valid_loss, exactitud_test = modelo.evaluate(set_datos)
print("""{} imagenes analizadas 
El modelo clasifico {:,.0f} imagenes correctamente, {:,.2f}% de exactitud""".format(len(set_datos.filenames),
                                                        len(set_datos.filenames)*exactitud_test,
                                                        exactitud_test*100,) )

**APLICAR FINE TUNNING AL MODELO**

In [ ]:
#@title Desabilitar y mostrar numero de capas del modelo
# Se desabilita el todo el modelo base para poder ser modificado
modelo_base.trainable = True
print("Numero de capas del modelo base: {}".format(
    len(modelo_base.layers)))


Numero de capas del modelo base: 707


In [ ]:
#@title Congelar Capas 
capas_congelar =  600#@param {type:"number"}
for layer in modelo_base.layers[:capas_congelar]:
    layer.trainable = False
print("Se congelaron las primeras {} capas del modelo base".format(capas_congelar) )

Se congelaron las primeras 600 capas del modelo base


In [ ]:
#@title Entrenar nuevamente el modelo
Entrenamiento_ft =  5#@param {type:"number"}

modelo.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

modelo.fit(imagenes_train,  
                    epochs=Entrenamiento_ft, 
                    validation_data=imagenes_test)

In [ ]:
#@title Resultados imagenes Test - Fine Tunning
valid_loss, exactitud_test = modelo.evaluate(valid_generator)
print("El modelo tiene una exactitud del {:,.2f}% con las imagenes de prueba ".format(exactitud_test*100) )

Pruebas con imagenes


In [ ]:
#@title Procesar imagen
# Identificar imagenes 
import numpy as np
from google.colab import files
from keras.preprocessing import image

#Predecir imagenes 
uploaded = files.upload()
for fn in uploaded.keys():   
  path = fn
  img = image.load_img(path, target_size=(imagen_size,imagen_size) )
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x= x/255
  images = np.vstack([x])
  classes = modelo.predict(x, batch_size=batch_size)

print("Probabilidad Gato: {}%".format(np.round(classes[:,0].flatten()*100, 2)) )
print("Probabilidad Perro: {}%".format(np.round(classes[:,1].flatten()*100, 2)) )

# Clasificación healt

In [ ]:
#@title Cargar Dataset
#CARGAR DATOS DESDE GCS
path_storage="001_storage/Datasets/"
archivo = "Neumonia.zip"
!gsutil cp gs://{path_storage}{archivo} /content/{archivo}

#DESCOMPRIMIR DATASET 
print("Descomprimiento archivo ...")
dataset_path = "./{}".format(archivo)
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()
print("Proceso Finalizado con exito !!!")

In [ ]:
#@title Inicializar variables 
imagen_size = 150
batch_size = 126
color_mode = "grayscale"

In [ ]:
#@title Cargar imagenes
#https://keras.io/api/preprocessing/image/
#DIRECTORIO DE LOS DATOS 
dataset_path_new = "./chest_xray/"
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "test")

#ESCALAR LAS IMAGENES 
#data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_train = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

data_gen_valid = ImageDataGenerator(rescale=1/255.)

#ESTANDARIZAR TAMAÑO DE LAS IMAGENES 
print("Imagenes de entrenamiento detectadas:")
imagenes_train = data_gen_train.flow_from_directory(
    train_dir, 
    target_size=(imagen_size,imagen_size),
    batch_size=batch_size, 
    color_mode=color_mode,
    class_mode="categorical")
    
print("\nImagenes de prueba detectadas:")
imagenes_test = data_gen_valid.flow_from_directory(
    validation_dir,
    target_size=(imagen_size,imagen_size), 
    batch_size=batch_size,
    color_mode=color_mode,
    class_mode="categorical")

print("""\n- Nombre imagenes de Entrenamiento: imagenes_train
- Nombre imagenes de prueba: imagenes_test""") 

In [ ]:
#@title Arquitectura por transferencia 
#CARGAR MODELO BASE 
tamanio_imagen = (imagen_size, imagen_size, 3)
modelo_base = tf.keras.applications.DenseNet201(
    input_shape=tamanio_imagen,
    include_top=False, 
    weights="imagenet")

#DEFINIR CAPAS QUE SE VAN AGREGAR AL MODELO BASE
modelo_base.trainable = False  #Congelar modelo Base
modelo_head = tf.keras.layers.GlobalAveragePooling2D()(modelo_base.output) #Agregamos capa AveragePooling 
modelo_head = tf.keras.layers.Flatten()(modelo_head)
modelo_head = tf.keras.layers.Dense(128, activation='relu')(modelo_head)
modelo_head = tf.keras.layers.Dropout(0.5)(modelo_head)
modelo_head = tf.keras.layers.Dense(units=2, activation='softmax')(modelo_head)

#AGREGAR CAPAS AL MODELO BASE
modelo = tf.keras.models.Model(inputs=modelo_base.input, outputs=modelo_head)
modelo.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss="categorical_crossentropy", metrics=["accuracy"])

modelo.summary() #Resumen de la nueva arquitectura Red Neuronal  

In [ ]:
#@title
modelo_base.trainable = True
print("Numero de capas del modelo base: {}".format(
    len(modelo_base.layers)))


In [ ]:
capas_congelar =  600#@param {type:"number"}
for layer in modelo_base.layers[:capas_congelar]:
    layer.trainable = False
print("Se congelaron las primeras {} capas del modelo base".format(capas_congelar) )

In [ ]:
#@title Arquitectura CNN
#Arquitectura de la Red Neuronal Convolucional (Ajustar imagen a 150x150 pixeles)
modelo = tf.keras.models.Sequential([
    # El tamaño de la imagen de entrada debe ser de 150x150 pixeles con 3 bytes color (input_shape)
    # Primera capa de convolucion con 64 filtros de tamaño 3x3 y capa MaxPooling tamaño 2x2 (Función activación: Rectificador Lineal)
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(imagen_size, imagen_size, 1)), 
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Segunda capa de convolución  con 64 filtros de tamaño 3x3 y capa MaxPooling tamaño 2x2 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Tercera capa de convolución  con 128 filtros de tamaño 3x3 y capa MaxPooling tamaño 2x2
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Cuarta capa de convolución  con 128 filtros de tamaño 3x3 y capa MaxPooling tamaño 2x2
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Capa de aplando para la entrada a la red neuronal y Dropout para evitar overfitting 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    
    # 512 neuronas de capa oculta y funcion de salida softmax (Clasificará 3 tipos de imagenes)
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

#Documentación del RMSprop: https://keras.io/api/optimizers/rmsprop/
modelo.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
               loss = 'categorical_crossentropy', metrics=['accuracy'])
modelo.summary()

In [ ]:
#@title Entrenar a la Red Neuronal
Entrenamiento =  25#@param {type:"number"}

!rm -rf ./logs/ #Limpiar datos TensorBoard 
modelo.fit(imagenes_train,
           validation_data=imagenes_test,
           epochs=Entrenamiento,            
           callbacks=[tensorboard_callback])

In [ ]:
#@title Procesar imagen
# Identificar imagenes 
import numpy as np
from google.colab import files
from keras.preprocessing import image

#Cargar y Predecir imagenes 
uploaded = files.upload()
for fn in uploaded.keys():   
  path = fn
  img = image.load_img(path,
                       target_size=(imagen_size,imagen_size),
                       color_mode=color_mode)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x= x/255
  #imagen = np.vstack([x])
  classes = modelo.predict(x,batch_size=imagen_size)

print("Probabilidad normal: {}%".format(np.round(classes[:,0].flatten()*100, 2)) )
print("Probabilidad neumonia: {}%".format(np.round(classes[:,1].flatten()*100, 2)) )

In [ ]:
#@title Cargar pesos del modelo entrenado
Archivo_modelo = "09165_Neumonia.h5" #@param {type:"string"}

#from keras.models import load_model
path_storage="001_storage/modelos/Healt/Naumonia/"
!gsutil cp gs://{path_storage}{Archivo_modelo} /content/{Archivo_modelo}

modelo.load_weights(Archivo_modelo)
print("Modelo entrenado cagado exitosamente !!!")

In [ ]:
#@title Evaluar exactitud del modelo
set_datos=imagenes_test
valid_loss, exactitud_test = modelo.evaluate(set_datos)
print("""{} imagenes analizadas 
El modelo clasifico {:,.0f} imagenes correctamente, {:,.2f}% de exactitud""".format(len(set_datos.filenames),
                                                        len(set_datos.filenames)*exactitud_test,
                                                        exactitud_test*100,) )